In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint
num_classes = 10
epochs = 10

In [ ]:
train_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_train.csv',sep=',')

test_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_test.csv', sep = ',')

In [ ]:
train_df.head()

In [ ]:
train_data = np.array(train_df, dtype = 'float32')

In [ ]:
test_data = np.array(test_df, dtype='float32')

In [ ]:
x_train = train_data[:,1:]/255
y_train = train_data[:,0]
x_test= test_data[:,1:]/255
y_test=test_data[:,0]

In [ ]:
x_train -= 0.5
x_test -= 0.5
x_train *= 2
x_test *= 2

In [ ]:
image_rows = 28

image_cols = 28

batch_size = 512

image_shape = (image_rows,image_cols,1)

In [ ]:
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.1,random_state = 12345)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],*image_shape)
x_test = x_test.reshape(x_test.shape[0],*image_shape)
x_validate = x_validate.reshape(x_validate.shape[0],*image_shape)
y_train2 = keras.utils.to_categorical(y_train).astype('float32')
y_validate2 = keras.utils.to_categorical(y_validate).astype('float32')

In [ ]:
num_classes = 10
model3=Sequential()
model3.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", 
          input_shape=image_shape, activation='relu'))
model3.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.5))
model3.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation='relu'))
model3.add(Conv2D(filters=256, kernel_size=(3, 3), padding="valid", activation='relu'))
model3.add(MaxPooling2D(pool_size=(3, 3)))
model3.add(Dropout(0.5))
model3.add(Flatten())
model3.add(Dense(256))
model3.add(LeakyReLU())
model3.add(Dropout(0.5))
model3.add(Dense(256))
model3.add(LeakyReLU())
#model2.add(Dropout(0.5))
model3.add(Dense(num_classes, activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

In [ ]:
history = model3.fit(
    x_train,
    y_train2,
    batch_size=500,
    epochs=50,
    verbose=1,
    validation_data=(x_validate,y_validate2)
)

In [ ]:
y_test2 = keras.utils.to_categorical(y_test).astype('float32')
score = model3.evaluate(x_test,y_test2,verbose=0)
print('Test Loss : {:.4f}'.format(score[0]))
print('Test Accuracy : {:.4f}'.format(score[1]))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes = model3.predict_classes(x_test)
#get the indices to be plotted
y_true = test_df.iloc[:, 0]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

In [ ]:
test = pd.read_csv('/kaggle/input/datamnist/test_data.csv', sep = ',')
test_d = np.array(test, dtype='float32')


In [ ]:
x_t = test_d[:,1:]/255
x_t -= 0.5
x_t *=2
x = x_t.reshape(x_t.shape[0],*image_shape)
p = model3.predict_classes(x)
print(p)

In [ ]:
pd.DataFrame(p, columns=['label']).to_csv('p.csv')
